<a href="https://colab.research.google.com/github/jyoti246/alexnet_cuda_cudnn/blob/main/Copy_of_cuda_alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-8c4e2njt
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-8c4e2njt
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=5d849835507dc0a857182b48791de408621c77af84531f84fad08e5d0a807e80
  Stored in directory: /tmp/pip-ephem-wheel-cache-5entshwc/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [ ]:
!git clone https://github.com/alessandrobessi/cuda-lab.git

Cloning into 'cuda-lab'...
remote: Enumerating objects: 56, done.
remote: Total 56 (delta 0), reused 0 (delta 0), pack-reused 56
Unpacking objects: 100% (56/56), done.


In [ ]:
!chmod 755 cuda-lab/INSTALL.sh

In [ ]:
!./cuda-lab/INSTALL.sh

35 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2021-04-12 19:35:26--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2021-04-12 19:35:26--  https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?ge8TEWN9FQ1

In [ ]:
import os
os.environ['PATH'] += ':/usr/local/cuda/bin'

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name alex.cu
#include <cudnn.h>
#include <cublas_v2.h>
#include <cuda_runtime.h>
#include <cstdlib>
#include <cassert>
#include <cstdlib>
#include <iostream>
#include <string>
#include <random>
#include <cmath>
#include <stdio.h>
#include <bits/stdc++.h>
#include <cuda_runtime.h>

using namespace std;

#define BATCH_SIZE 16
#define MAX_THREADS_PER_BLOCK 1024 // according to GTX 1050 Ti
#define LAYERS 5
#define COMP_TIME 3
#define EXEC_TIME 1
#define NUM_BATCHES 2

int roundUp(int num, int den)
{

  return((num + den - 1 )/(den));

}

struct convDim_t{

  int Height;
  int Width;
  int Channels;
  int Batch;
};

struct kernelDim_t{

  int kernelSize;
  int kernelHeight;
  int kernelWidth;
  int strideHeight;
  int strideWidth;
  int padHeight;
  int padWidth;
  int dilationHeight;
  int dilationWidth;
};


convDim_t setConvSpecs(int ht, int wd, int ch, int bt){

  convDim_t temp;
  temp.Height = ht;
  temp.Width = wd;
  temp.Channels = ch;
  temp.Batch = bt;

  return temp;
}

kernelDim_t setKernelSpecs(int size, int fheight, int fwidth, int sheight, int swidth, int pheight, int pwidth, int dheight, int dwidth){

  kernelDim_t layerKernel;
  layerKernel.kernelSize = size;
  layerKernel.kernelHeight = fheight;
  layerKernel.kernelWidth = fwidth;
  layerKernel.strideHeight = sheight;
  layerKernel.strideWidth = swidth;
  layerKernel.padHeight = pheight;
  layerKernel.padWidth = pwidth;
  layerKernel.dilationHeight = dheight;
  layerKernel.dilationWidth = dwidth;

  return layerKernel;
}



#define checkCUDNN(expression)                             \
{                                                          \
  cudnnStatus_t status = (expression);                     \
  if (status != CUDNN_STATUS_SUCCESS) {                    \
    std::cerr << "Error on line " << __LINE__ << ": "      \
              << cudnnGetErrorString(status) << std::endl; \
    std::exit(EXIT_FAILURE);                               \
  }                                                        \
}


float alpha = 1.0;
float beta = 0.0;

class ConvLayers{
    
public:
    float *kernelTensor{nullptr};		
    int layerIndex;
	  size_t workspace_bytes{0};

    convDim_t outDims;
    convDim_t inDims;
    kernelDim_t kernelDims;
	    

    cudnnTensorDescriptor_t input_descriptor;
    cudnnTensorDescriptor_t output_descriptor;
    cudnnFilterDescriptor_t kernel_descriptor;
    cudnnConvolutionDescriptor_t convolution_descriptor;
    cudnnConvolutionFwdAlgo_t convolution_algorithm;

    ConvLayers(){}

    ConvLayers(int index, convDim_t inDim, kernelDim_t kdims, convDim_t outDims){

      this->inDims = inDim;
      this->kernelDims = kdims;
      this->layerIndex = index;
	    this->outDims = outDims;
    }

    void set(int index, convDim_t inDim, kernelDim_t kdims, convDim_t outDims){

      this->inDims = inDim;
      this->kernelDims = kdims;
      this->layerIndex = index;
      this->outDims = outDims;
    }

    void buildConvLayer();

    void fwdProp(cudaStream_t stream, cudnnHandle_t cudnn, float *inputTensor, float* &outputTensor, void* &d_workspace);

};

void ConvLayers::buildConvLayer(){
	checkCUDNN(cudnnCreateTensorDescriptor(&input_descriptor));
	checkCUDNN(cudnnSetTensor4dDescriptor(input_descriptor,
										/*format=*/CUDNN_TENSOR_NHWC,
										/*dataType=*/CUDNN_DATA_FLOAT,
										/*batch_size=*/inDims.Batch,
										/*channels=*/inDims.Channels,
										/*image_height=*/inDims.Height,
										/*image_width=*/inDims.Width));



	checkCUDNN(cudnnCreateTensorDescriptor(&output_descriptor));
	checkCUDNN(cudnnSetTensor4dDescriptor(output_descriptor,
										/*format=*/CUDNN_TENSOR_NHWC,
										/*dataType=*/CUDNN_DATA_FLOAT,
										/*batch_size=*/outDims.Batch,
										/*channels=*/outDims.Channels,
										/*image_height=*/outDims.Height,
										/*image_width=*/outDims.Width));   
										
										

	checkCUDNN(cudnnCreateFilterDescriptor(&kernel_descriptor));
	checkCUDNN(cudnnSetFilter4dDescriptor(kernel_descriptor,
										/*dataType=*/CUDNN_DATA_FLOAT,
										/*format=*/CUDNN_TENSOR_NCHW,
										/*out_channels=*/outDims.Channels,
										/*in_channels=*/inDims.Channels,
										/*kernel_height=*/kernelDims.kernelHeight,
										/*kernel_width=*/kernelDims.kernelWidth)); 
										
										

	checkCUDNN(cudnnCreateConvolutionDescriptor(&convolution_descriptor));
	checkCUDNN(cudnnSetConvolution2dDescriptor(convolution_descriptor,
										/*pad_height=*/kernelDims.padHeight,
										/*pad_width=*/kernelDims.padWidth,
										/*vertical_stride=*/kernelDims.strideHeight,
										/*horizontal_stride=*/kernelDims.strideWidth,
										/*dilation_height=*/kernelDims.dilationHeight,
										/*dilation_width=*/kernelDims.dilationWidth,
										/*mode=*/CUDNN_CROSS_CORRELATION,
										/*computeType=*/CUDNN_DATA_FLOAT));


	float h_kernel[outDims.Channels][inDims.Channels][kernelDims.kernelHeight][kernelDims.kernelWidth];
    for (int kernel = 0; kernel < outDims.Channels; ++kernel) {
    for (int channel = 0; channel < inDims.Channels; ++channel) {
        for (int row = 0; row < kernelDims.kernelHeight; ++row) {
        for (int column = 0; column < kernelDims.kernelWidth; ++column) {
            h_kernel[kernel][channel][row][column] = 0.5;
        }
        }
    }
    }

    cudaMalloc(&kernelTensor, sizeof(h_kernel));
    cudaMemcpy(kernelTensor, h_kernel, sizeof(h_kernel), cudaMemcpyHostToDevice); 
}


void ConvLayers::fwdProp(cudaStream_t stream, cudnnHandle_t cudnn, float *inputTensor, float* &outputTensor, void* &d_workspace)
{
	checkCUDNN(cudnnGetConvolutionForwardAlgorithm(cudnn,
                                            input_descriptor,
                                            kernel_descriptor,
                                            convolution_descriptor,
                                            output_descriptor,
                                            CUDNN_CONVOLUTION_FWD_PREFER_FASTEST,
                                            /*memoryLimitInBytes=*/0,
                                            &convolution_algorithm));
                                            

    checkCUDNN(cudnnGetConvolutionForwardWorkspaceSize(cudnn,
                                                    input_descriptor,
                                                    kernel_descriptor,
                                                    convolution_descriptor,
                                                    output_descriptor,
                                                    convolution_algorithm,
                                                    &workspace_bytes));                  
                
    cudaMalloc(&d_workspace, workspace_bytes);       

	int out_bytes = outDims.Batch*outDims.Channels*outDims.Height*outDims.Width*sizeof(float);
    cudaMalloc(&outputTensor, out_bytes);
    cudaMemsetAsync(outputTensor, 0, out_bytes, stream);


    checkCUDNN(cudnnConvolutionForward(cudnn,
                                   &alpha,
                                   input_descriptor,
                                   inputTensor,
                                   kernel_descriptor,
                                   kernelTensor,
                                   convolution_descriptor,
                                   convolution_algorithm,
                                   d_workspace,
                                   workspace_bytes,
                                   &beta,
                                   output_descriptor,
                                   outputTensor));

}

ConvLayers convlayer1;
ConvLayers convlayer2;
ConvLayers convlayer3;
ConvLayers convlayer4;
ConvLayers convlayer5;


float img[BATCH_SIZE][224][224][3];
float output1[BATCH_SIZE][224][224][10];
float output2[BATCH_SIZE][224][224][30];
float output3[BATCH_SIZE][224][224][30];
float output4[BATCH_SIZE][224][224][10];
float output5[BATCH_SIZE][224][224][3];

class Kernel{

  public :
    cudaStream_t stream;
    cublasHandle_t cublas;
    cudnnHandle_t cudnn;
    int id;
    int layer;
    double tbRatio;
    // float ** twoDInput;
    // float ** twoDOutput;
    double execTime;
    double compTime;
    

    Kernel(){}
    
    Kernel(cudaStream_t stream,
        cublasHandle_t cublas,
        cudnnHandle_t cudnn,
        int layerIndex,
        double startTime,
        double deadTime,
        double tbratio,
        int id){
            this->stream = stream;
            this->cublas = cublas;
            this->cudnn = cudnn;
            this->layer = layerIndex;
            this->execTime = deadTime - startTime;
            this->compTime = LAYERS*COMP_TIME;
            this->tbRatio = tbratio;
            this->id = id;
        }
    
    Kernel(cudaStream_t stream,
        cublasHandle_t cublas,
        cudnnHandle_t cudnn,
        int layerIndex,
        float **twoDInput,
        double startTime,
        double deadTime,
        double tbratio,
        int id){
            this->stream = stream;
            this->cublas = cublas;
            this->cudnn = cudnn;
            this->layer = layerIndex;
            // this->twoDInput = twoDInput;
            this->execTime = deadTime - startTime;
            this->compTime = LAYERS*COMP_TIME;
            this->tbRatio = tbratio;
            this->id =id;
        }
    
    void update()
    {
        layer++;
        compTime -= COMP_TIME;
        // TODO update tbratio properly
        if(layer >= 7){
            tbRatio = 0.2;
            // twoDInput = twoDOutput;
        }
        // else
        // {
        //     fourDInput = fourDOutput;
        // }
    }


};

void execute(vector<Kernel> C);


typedef pair<int, Kernel> Node;

struct cmp
{
  bool operator()(const Node &A,const Node &B) const
  {
    if(A.first != B.first)
            return A.first < B.first;
        else
            return A.second.execTime - A.second.compTime < B.second.execTime - B.second.compTime;
  }
};

typedef set<Node, cmp> SET;

bool compareKernel(Kernel k1, Kernel k2){
    return k1.compTime < k2.compTime;
}

int getSlackTime(int compTime, int execTime){
    return execTime - compTime;
}



vector<Kernel> G;

SET enqueue(SET Queue, vector<Kernel> k){
    SET newQueue;
    for(SET::iterator it=Queue.begin();it!=Queue.end();it++){
        Kernel temp_kernel = it->second;
        temp_kernel.execTime -= EXEC_TIME;
        newQueue.insert({getSlackTime(temp_kernel.compTime,temp_kernel.execTime),temp_kernel});
    }
    for(int i=0;i<k.size();i++){
        if(k[i].layer == LAYERS+1)
            continue;
        k[i].execTime -= EXEC_TIME;
        cout<<"Enqueue ID: "<<k[i].id<<" "<<k[i].layer<<endl;
        newQueue.insert({getSlackTime(k[i].compTime, k[i].execTime), k[i]});
    }
    return newQueue;
}


SET dequeue(SET& Queue){
    SET newQueue;
    
    cout<<"Printing Queue"<<endl;
    cout<<Queue.size()<<endl;
    for(SET:: iterator it=Queue.begin();it!=Queue.end();it++){
        Kernel temp_kernel = it->second;
        cout<<"Dequeu ID: "<<temp_kernel.id<<" "<<temp_kernel.layer<<endl;
        temp_kernel.execTime -= COMP_TIME;
        newQueue.insert({getSlackTime(temp_kernel.compTime,temp_kernel.execTime),temp_kernel});
    }
    Queue.clear();
    vector<Kernel> C;
    if(G.size()>0){
        execute(G);
        cout<<"Execution size "<<G.size()<<endl;
    }
        
    for(int i=0;i<G.size();i++){
        cout<<"EXEC Layer: "<<G[i].layer<<endl;
        cout<<"Priority:"<<G[i].execTime - G[i].compTime<<endl;
        if(G[i].layer != LAYERS){
            G[i].update();
        }
    }
    newQueue = enqueue(newQueue, G);
    G.clear();
    Kernel temp = (*(newQueue.begin())).second;
    newQueue.erase(newQueue.begin());
    G.push_back(temp);
    double total = 0;
    total += temp.tbRatio;
    if(total < 1){
        for(SET::iterator it = newQueue.begin();it!=newQueue.end();){
            if((*it).second.tbRatio<1){
                Kernel temp2 = (*it).second;
                total += temp2.tbRatio;
                it = newQueue.erase(it);
                G.push_back(temp2);
            }
            else
            {
                it++;
            }
            cout<<"Total TBratio "<<total<<endl;
            if(total > 1)
                break;
        }
    }
    else
    {
        cout<<"Single kernel execution"<<endl;
        execute(G);
        for(int i=0;i<G.size();i++){
            cout<<"Priority:"<<G[i].execTime - G[i].compTime<<endl;
            if(G[i].layer != LAYERS){
                G[i].update();
            }
            else
                G[i].layer++;
        }
        newQueue = enqueue(newQueue, G);
        G.clear();

        return newQueue;
    }

    execute(G);
    cout<<"Execution size: "<<G.size()<<endl;
    for(int i=0;i<G.size();i++){
        cout<<"Priority:"<<G[i].execTime - G[i].compTime<<endl;
        if(G[i].layer != LAYERS){
            G[i].update();
        }
        else
        {
            G[i].layer++;
        }
    }
    newQueue = enqueue(newQueue, G);
    G.clear();
    return newQueue;   
}

void call_conv1(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][3], float output[BATCH_SIZE][224][224][10])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV1 DONE" << endl;
}

void call_conv2(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][10], float output[BATCH_SIZE][224][224][30])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV2 DONE" << endl;
}


void call_conv3(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][30], float output[BATCH_SIZE][224][224][30])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV3 DONE" << endl;
}

void call_conv4(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][30], float output[BATCH_SIZE][224][224][10])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV4 DONE" << endl;
}

void call_conv5(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float input[][224][224][10], float output[BATCH_SIZE][224][224][3])
{
    float *inputTensor;
	float *outputTensor;
	int inp_size = convlayer1.inDims.Height * convlayer1.inDims.Width * convlayer1.inDims.Channels * convlayer1.inDims.Batch * sizeof(float);
    cudaMalloc(&inputTensor, inp_size);
    cudaMemcpyAsync(inputTensor, input, inp_size, cudaMemcpyHostToDevice, stream);

    void *d_workspace;
    convlayer1.fwdProp(stream, CUDNN, inputTensor, outputTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch*sizeof(float);
    
    cudaMemcpyAsync(output, outputTensor, out_size, cudaMemcpyDeviceToHost, stream);

    cout << "CONV5 DONE" << endl;
}




void execute(vector<Kernel> C)
{
    
    for(int i=0; i<C.size();i++){
        Kernel K = C[i];
        if(K.layer == 1)
            call_conv1(convlayer1, K.stream, K.cudnn, img, output1);
        else if(K.layer == 2)
            call_conv2(convlayer2, K.stream, K.cudnn, output1, output2);
        else if(K.layer == 3)
            call_conv3(convlayer3, K.stream, K.cudnn, output2, output3);
        else if(K.layer == 4)
            call_conv4(convlayer4, K.stream, K.cudnn, output3, output4);
        else
            call_conv5(convlayer5, K.stream, K.cudnn, output4, output5);
    }
	
}

//channel, height, width
int main(){
  for(int i=0;i<BATCH_SIZE;++i){
    for(int j=0; j<224;++j){
      for(int k=0;k<224;++k){
        for(int l=0;l<3;++l){
          img[i][j][k][l]=0.001;
        }
      }
    }
  }

	convDim_t InputDims1 = setConvSpecs(224, 224, 3, BATCH_SIZE);
	kernelDim_t layerKernel1 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims1 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	convlayer1.set(1, InputDims1, layerKernel1, OutputDims1);
	convlayer1.buildConvLayer();
	
	convDim_t InputDims2 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	kernelDim_t layerKernel2 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims2 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	convlayer2.set(2, InputDims2, layerKernel2, OutputDims2);
	convlayer2.buildConvLayer();

	convDim_t InputDims3 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	kernelDim_t layerKernel3 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims3 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	convlayer3.set(3, InputDims3, layerKernel3, OutputDims3);
	convlayer3.buildConvLayer();

	convDim_t InputDims4 = setConvSpecs(224, 224, 30, BATCH_SIZE);
	kernelDim_t layerKernel4 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims4 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	convlayer4.set(4, InputDims4, layerKernel4, OutputDims4);
	convlayer4.buildConvLayer();

	convDim_t InputDims5 = setConvSpecs(224, 224, 10, BATCH_SIZE);
	kernelDim_t layerKernel5 = setKernelSpecs(0,3,3,1,1,1,1,1,1);
	convDim_t OutputDims5 = setConvSpecs(224, 224, 3, BATCH_SIZE);
	convlayer5.set(5, InputDims5, layerKernel5, OutputDims5);
	convlayer5.buildConvLayer();

	int nstreams = 4;
	cudaStream_t stream[nstreams];
	cudnnHandle_t cudnn[nstreams];
    cublasHandle_t cublas[nstreams];

  

	for(int i=0;i<nstreams;++i){
		cudaStreamCreate(&stream[i]);
		checkCUDNN(cudnnCreate(&cudnn[i]));
		cudnnSetStream(cudnn[i], stream[i]);
        cublasSetStream(cublas[i], stream[i]);
	}

for(int j=1;j<=nstreams;j++){
    vector<Kernel> inp;
    SET pq;

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    cudaEventRecord(start);
    
    for(int i=0;i<NUM_BATCHES;i++)
    {
          Kernel k(stream[i%j], cublas[i%j], cudnn[i%j], 1, 0, 30+i, 1.2, i);   
          inp.push_back(k);
    }
      pq = enqueue(pq, inp);
      while(!pq.empty()){
          pq = dequeue(pq);
      }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float elapsedtime;
    cudaEventElapsedTime(&elapsedtime, start, stop);
    cout<<"Num Streams"<<j << "ELAPSED TIME: " << elapsedtime << endl;
    cudaDeviceSynchronize();

}
    

    cout<<output1[0][0][0][0]<<endl;
    cout<<output2[0][0][0][0]<<endl;
    cout<<output3[0][0][0][0]<<endl;
    cout<<output4[0][0][0][0]<<endl;
    cout<<output5[0][0][0][0]<<endl;
	return 0;
}



'File written in /content/src/alex.cu'

In [ ]:
!nvcc /content/src/alex.cu `pkg-config --cflags --libs opencv` -lcudnn -lcublas -lopencv_imgcodecs -lopencv_imgproc -lopencv_core -pg -std=c++11 -o /content/alex

In [ ]:
!/content/alex 

Enqueue ID: 0 1
Enqueue ID: 1 1
Printing Queue
2
Dequeu ID: 0 1
Dequeu ID: 1 1
Single kernel execution
CONV1 DONE
Priority:10
Enqueue ID: 0 2
Printing Queue
2
Dequeu ID: 1 1
Dequeu ID: 0 2
Single kernel execution
CONV1 DONE
Priority:6
Enqueue ID: 1 2
Printing Queue
2
Dequeu ID: 0 2
Dequeu ID: 1 2
Single kernel execution
CONV2 DONE
Priority:3
Enqueue ID: 0 3
Printing Queue
2
Dequeu ID: 1 2
Dequeu ID: 0 3
Single kernel execution
CONV2 DONE
Priority:-1
Enqueue ID: 1 3
Printing Queue
2
Dequeu ID: 0 3
Dequeu ID: 1 3
Single kernel execution
CONV3 DONE
Priority:-4
Enqueue ID: 0 4
Printing Queue
2
Dequeu ID: 1 3
Dequeu ID: 0 4
Single kernel execution
CONV3 DONE
Priority:-8
Enqueue ID: 1 4
Printing Queue
2
Dequeu ID: 0 4
Dequeu ID: 1 4
Single kernel execution
CONV4 DONE
Priority:-11
Enqueue ID: 0 5
Printing Queue
2
Dequeu ID: 1 4
Dequeu ID: 0 5
Single kernel execution
CONV4 DONE
Priority:-15
Enqueue ID: 1 5
Printing Queue
2
Dequeu ID: 0 5
Dequeu ID: 1 5
Single kernel execution
CONV5 DONE
Priori

In [ ]:
! lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

!

In [ ]:
!nvprof /content/alex

Output image size 1 X 28 X 28 X 64
==3684== NVPROF is profiling process 3684, command: /content/alex
Output image size 1 X 14 X 14 X 192
Output image size 1 X 14 X 14 X 384
Output image size 1 X 14 X 14 X 256
Output image size 1 X 7 X 7 X 256
Output image size 4096
Output image size 4096
Output image size 1000
BUILT ALL LAYERS
K1:0.6 21 50
Printing Queue
ID: 0
ID: 1
ID: 2
ID: 3
ID: 4
ID: 5
ID: 6
ID: 7
ID: 8
ID: 9
Total TBratio 1.2
CONV1 DONE
CONV1 DONE
Execution size: 2
Priority:15
Priority:16
Printing Queue
ID: 2
ID: 3
ID: 4
ID: 5
ID: 6
ID: 7
ID: 8
ID: 9
Total TBratio 1.2
CONV1 DONE
CONV1 DONE
Execution size: 2
Priority:12
Priority:13
Printing Queue
ID: 4
ID: 5
ID: 6
ID: 7
ID: 8
ID: 9
Total TBratio 1.2
CONV1 DONE
CONV1 DONE
Execution size: 2
Priority:9
Priority:10
Printing Queue
ID: 6
ID: 7
ID: 8
ID: 9
Total TBratio 1.2
CONV1 DONE
CONV1 DONE
Execution size: 2
Priority:6
Priority:7
Printing Queue
ID: 8
ID: 9
ID: 7
Total TBratio 1.2
CONV1 DONE
CONV1 DONE
Execution size: 2
Priority:3
Pri

In [ ]:
# %%cuda --name yolo.cu
# #include<bits/stdc++.h>
# using namespace std;

# // Initialize the parameters
# float confThreshold = 0.5; // Confidence threshold
# float nmsThreshold = 0.4;  // Non-maximum suppression threshold
# int inpWidth = 416;        // Width of network's input image
# int inpHeight = 416;       // Height of network's input image

# // Load names of classes
# string classesFile = "/content/drive/My Drive/hp3_data/cfg/coco.names";
# ifstream ifs(classesFile.c_str());
# string line;
# while (getline(ifs, line)) classes.push_back(line);

# // Give the configuration and weight files for the model
# String modelConfiguration = "/content/drive/My Drive/hp3_data/cfg/yolov3.cfg";
# String modelWeights = "/content/drive/My Drive/hp3_data/cfg/yolov3.weights";

# // Load the network
# Net net = readNetFromDarknet(modelConfiguration, modelWeights);
# net.setPreferableBackend(DNN_BACKEND_OPENCV);
# net.setPreferableTarget(DNN_TARGET_CPU);

# outputFile = "/content/drive/My Drive/hp3_data/yolo_out_cpp.avi";
# if (parser.has("image"))
# {
#     // Open the image file
#     str = parser.get<String>("image");
#     ifstream ifile(str);
#     if (!ifile) throw("error");
#     cap.open(str);
#     str.replace(str.end()-4, str.end(), "_yolo_out_cpp.jpg");
#     outputFile = str;
# }
# else if (parser.has("video"))
# {
#     // Open the video file
#     str = parser.get<String>("video");
#     ifstream ifile(str);
#     if (!ifile) throw("error");
#     cap.open(str);
#     str.replace(str.end()-4, str.end(), "_yolo_out_cpp.avi");
#     outputFile = str;
# }
# // Open the webcaom
# else cap.open(parser.get<int>("device"));

# catch(...) {
#     cout << "Could not open the input image/video stream" << endl;
#     return 0;
# }

# // Get the video writer initialized to save the output video
# if (!parser.has("image")) {
#     video.open(outputFile, VideoWriter::fourcc('M','J','P','G'), 28, Size(cap.get(CAP_PROP_FRAME_WIDTH), cap.get(CAP_PROP_FRAME_HEIGHT)));
# }


# // Process frames.
# while (waitKey(1) < 0)
# {
#     // get frame from the video
#     cap >> frame;

#     // Stop the program if reached end of video
#     if (frame.empty()) {
#         cout << "Done processing !!!" << endl;
#         cout << "Output file is stored as " << outputFile << endl;
#         waitKey(3000);
#         break;
#     }
#     // Create a 4D blob from a frame.
#     blobFromImage(frame, blob, 1/255.0, cv::Size(inpWidth, inpHeight), Scalar(0,0,0), true, false);
        
#     //Sets the input to the network
#     net.setInput(blob);
    
#     // Runs the forward pass to get output of the output layers
#     vector<Mat> outs;
#     net.forward(outs, getOutputsNames(net));
        
#     // Remove the bounding boxes with low confidence
#     postprocess(frame, outs);
        
#     // Put efficiency information. The function getPerfProfile returns the overall time for inference(t) and the timings for each of the layers(in layersTimes)
#     vector<double> layersTimes;
#     double freq = getTickFrequency() / 1000;
#     double t = net.getPerfProfile(layersTimes) / freq;
#     string label = format("Inference time for a frame : %.2f ms", t);
#     putText(frame, label, Point(0, 15), FONT_HERSHEY_SIMPLEX, 0.5, Scalar(0, 0, 255));
        
#     // Write the frame with the detection boxes
#     Mat detectedFrame;
#     frame.convertTo(detectedFrame, CV_8U);
#     if (parser.has("image")) imwrite(outputFile, detectedFrame);
#     else video.write(detectedFrame);
        
#     imshow(kWinName, frame);
        
# }

# // Get the names of the output layers
# vector<String> getOutputsNames(const Net& net)
# {
#     static vector<String> names;
#     if (names.empty())
#     {
#         //Get the indices of the output layers, i.e. the layers with unconnected outputs
#         vector<int> outLayers = net.getUnconnectedOutLayers();
        
#         //get the names of all the layers in the network
#         vector<String> layersNames = net.getLayerNames();
        
#         // Get the names of the output layers in names
#         names.resize(outLayers.size());
#         for (size_t i = 0; i < outLayers.size(); ++i)
#         names[i] = layersNames[outLayers[i] - 1];
#     }
#     return names;
# }


# // Remove the bounding boxes with low confidence using non-maxima suppression
# void postprocess(Mat& frame, const vector<Mat>& outs)
# {
#     vector<int> classIds;
#     vector<float> confidences;
#     vector<Rect> boxes;
    
#     for (size_t i = 0; i < outs.size(); ++i)
#     {
#         // Scan through all the bounding boxes output from the network and keep only the
#         // ones with high confidence scores. Assign the box's class label as the class
#         // with the highest score for the box.
#         float* data = (float*)outs[i].data;
#         for (int j = 0; j < outs[i].rows; ++j, data += outs[i].cols)
#         {
#             Mat scores = outs[i].row(j).colRange(5, outs[i].cols);
#             Point classIdPoint;
#             double confidence;
#             // Get the value and location of the maximum score
#             minMaxLoc(scores, 0, &confidence, 0, &classIdPoint);
#             if (confidence > confThreshold)
#             {
#                 int centerX = (int)(data[0] * frame.cols);
#                 int centerY = (int)(data[1] * frame.rows);
#                 int width = (int)(data[2] * frame.cols);
#                 int height = (int)(data[3] * frame.rows);
#                 int left = centerX - width / 2;
#                 int top = centerY - height / 2;
                
#                 classIds.push_back(classIdPoint.x);
#                 confidences.push_back((float)confidence);
#                 boxes.push_back(Rect(left, top, width, height));
#             }
#         }
#     }
    
#     // Perform non maximum suppression to eliminate redundant overlapping boxes with
#     // lower confidences
#     vector<int> indices;
#     NMSBoxes(boxes, confidences, confThreshold, nmsThreshold, indices);
#     for (size_t i = 0; i < indices.size(); ++i)
#     {
#         int idx = indices[i];
#         Rect box = boxes[idx];
#         drawPred(classIds[idx], confidences[idx], box.x, box.y,
#                  box.x + box.width, box.y + box.height, frame);
#     }
# }

# // Draw the predicted bounding box
# void drawPred(int classId, float conf, int left, int top, int right, int bottom, Mat& frame)
# {
#     //Draw a rectangle displaying the bounding box
#     rectangle(frame, Point(left, top), Point(right, bottom), Scalar(255, 178, 50), 3);
    
#     //Get the label for the class name and its confidence
#     string label = format("%.2f", conf);
#     if (!classes.empty())
#     {
#         CV_Assert(classId < (int)classes.size());
#         label = classes[classId] + ":" + label;
#     }
    
#     //Display the label at the top of the bounding box
#     int baseLine;
#     Size labelSize = getTextSize(label, FONT_HERSHEY_SIMPLEX, 0.5, 1, &baseLine);
#     top = max(top, labelSize.height);
#     rectangle(frame, Point(left, top - round(1.5*labelSize.height)), Point(left + round(1.5*labelSize.width), top + baseLine), Scalar(255, 255, 255), FILLED);
#     putText(frame, label, Point(left, top), FONT_HERSHEY_SIMPLEX, 0.75, Scalar(0,0,0),1);
# }


In [ ]:
%%cuda --name alex.cu
#include <cudnn.h>
#include <cublas_v2.h>
#include <cudaProfiler.h>
#include <cuda_runtime.h>
#include <cstdlib>
#include <cassert>
#include <cstdlib>
#include <iostream>
#include <string>
#include <random>
#include <cmath>
#include <stdio.h>
#include <bits/stdc++.h>
#include <cuda_runtime.h>

using namespace std;

#define BATCH_SIZE 1
#define OVERLAP_POOLING 1
#define BIAS_INIT_VAL 0.001
#define MAX_THREADS_PER_BLOCK 1024 // according to GTX 1050 Ti
#define LAYERS 7
#define CONST_TIME 3

int roundUp(int num, int den)
{

  return((num + den - 1 )/(den));

}

struct convDim_t{

  int Height;
  int Width;
  int Channels;
  int Batch;
};

struct poolDim_t{

  int Height;
  int Width;
  int padHeight;
  int padWidth;
  int strideHeight;
  int strideWidth;
};

struct kernelDim_t{

  int kernelSize;
  int kernelHeight;
  int kernelWidth;
  int strideHeight;
  int strideWidth;
  int padHeight;
  int padWidth;
  int dilationHeight;
  int dilationWidth;
};

convDim_t setConvSpecs(int ht, int wd, int ch, int bt){

  convDim_t temp;
  temp.Height = ht;
  temp.Width = wd;
  temp.Channels = ch;
  temp.Batch = bt;

  return temp;
}

kernelDim_t setKernelSpecs(int size, int fheight, int fwidth, int sheight, int swidth, int pheight, int pwidth, int dheight, int dwidth){

  kernelDim_t layerKernel;
  layerKernel.kernelSize = size;
  layerKernel.kernelHeight = fheight;
  layerKernel.kernelWidth = fwidth;
  layerKernel.strideHeight = sheight;
  layerKernel.strideWidth = swidth;
  layerKernel.padHeight = pheight;
  layerKernel.padWidth = pwidth;
  layerKernel.dilationHeight = dheight;
  layerKernel.dilationWidth = dwidth;

  return layerKernel;
}

poolDim_t setPoolSpecs(bool flagOverlap){

  poolDim_t poolDims;

  if(flagOverlap){
    poolDims.Height = 3;
    poolDims.Width = 3;
    poolDims.padHeight = 0;
    poolDims.padWidth = 0;
    poolDims.strideHeight = 2;
    poolDims.strideWidth = 2;  
  }
  else{
    poolDims.Height = 2;
    poolDims.Width = 2;
    poolDims.padHeight = 1;
    poolDims.padWidth = 1;
    poolDims.strideHeight = 2;
    poolDims.strideWidth = 2;
  }

  return poolDims;
  
}



#define checkCUDNN(expression)                             \
{                                                          \
  cudnnStatus_t status = (expression);                     \
  if (status != CUDNN_STATUS_SUCCESS) {                    \
    std::cerr << "Error on line " << __LINE__ << ": "      \
              << cudnnGetErrorString(status) << std::endl; \
    std::exit(EXIT_FAILURE);                               \
  }                                                        \
}


float alpha = 1.0;
float beta = 0.5;



class ConvLayers{
    
public:
    float *kernelTensor;
    float *biasTensor;		
    int layerIndex;
    float alph, bet;

    cudnnTensorFormat_t TensorFormat;
    cudnnDataType_t DataType;
    cudnnConvolutionMode_t ConvMode;
    cudnnActivationMode_t ActivationMode;
    cudnnPoolingMode_t PoolingMode;

    convDim_t outDims;
    convDim_t inDims;
    kernelDim_t kernelDims;
    poolDim_t poolDims;

    random_device rd{};
    mt19937 gen{rd()};  
    normal_distribution<> d{0,1}; 

    /**
    float* conv_output{nullptr}; // output of convolution operation
    float* poolTensor{nullptr};  // output of pooling layer, if exists
    void* d_workspace{nullptr};
    size_t workspaceBytes{0};
    **/

    

    int convOutDimHeight{0}, convOutDimWidth{0}, convOutDimChannels{0}, convOutDimBatchSize{0};
    int poolOutBatchSize{0}, poolOutChannels{0}, poolOutHeight{0}, poolOutWidth{0};

    bool POOL;  // True if pooling is to be done in this layer, otherwise False

    cudnnTensorDescriptor_t input_descriptor;
    cudnnFilterDescriptor_t kernel_descriptor;
    cudnnConvolutionDescriptor_t convolution_descriptor;
    cudnnTensorDescriptor_t bias_descriptor;
    cudnnTensorDescriptor_t convOutput_descriptor;
    cudnnConvolutionFwdAlgo_t convolution_algorithm;
    cudnnActivationDescriptor_t activation_descriptor;
    cudnnPoolingDescriptor_t pooling_descriptor;
    cudnnTensorDescriptor_t poolTensor_descriptor;

    ConvLayers(){}

    ConvLayers( int index, convDim_t inDim, kernelDim_t kdims, float a, float b, 
        cudnnTensorFormat_t t_format, cudnnDataType_t d_type, cudnnConvolutionMode_t c_mode, cudnnActivationMode_t ActMode){

        this->POOL = false;
        this->inDims = inDim;
        this->kernelDims = kdims;
        this->layerIndex = index;
        this->alph = a;
        this->bet = b;
        this->TensorFormat = t_format;
        this->DataType = d_type;			
        this->ConvMode = c_mode;
        this->ActivationMode = ActMode;
    }

    void set( int index, convDim_t inDim, kernelDim_t kdims, float a, float b, 
        cudnnTensorFormat_t t_format, cudnnDataType_t d_type, cudnnConvolutionMode_t c_mode, cudnnActivationMode_t ActMode){

        this->POOL = false;
        this->inDims = inDim;
        this->kernelDims = kdims;
        this->layerIndex = index;
        this->alph = a;
        this->bet = b;
        this->TensorFormat = t_format;
        this->DataType = d_type;			
        this->ConvMode = c_mode;
        this->ActivationMode = ActMode;
    }

    ConvLayers(int index, convDim_t inDim, kernelDim_t kdims, poolDim_t pdims, float a, float b, 
    cudnnTensorFormat_t t_format, cudnnDataType_t d_type, cudnnConvolutionMode_t c_mode, cudnnActivationMode_t ActMode,cudnnPoolingMode_t poolMode){

    this->POOL = true;
    this->inDims = inDim;
    this->kernelDims = kdims;
    this->poolDims = pdims;
    this->layerIndex = index;
    this->alph = a; 
    this->bet = b;
    this->TensorFormat = t_format;
    this->DataType = d_type;      
    this->ConvMode = c_mode;
    this->ActivationMode = ActMode;
    this->PoolingMode = poolMode; 
    }

    void set(int index, convDim_t inDim, kernelDim_t kdims, poolDim_t pdims, float a, float b, 
    cudnnTensorFormat_t t_format, cudnnDataType_t d_type, cudnnConvolutionMode_t c_mode, cudnnActivationMode_t ActMode,cudnnPoolingMode_t poolMode){

    this->POOL = true;
    this->inDims = inDim;
    this->kernelDims = kdims;
    this->poolDims = pdims;
    this->layerIndex = index;
    this->alph = a; 
    this->bet = b;
    this->TensorFormat = t_format;
    this->DataType = d_type;      
    this->ConvMode = c_mode;
    this->ActivationMode = ActMode;
    this->PoolingMode = poolMode; 
    }

    ConvLayers(const ConvLayers& old_layer){
        this->POOL = old_layer.POOL;
        this->inDims = old_layer.inDims;
        this->kernelDims = old_layer.kernelDims;
        this->poolDims = old_layer.poolDims;
        this->layerIndex = old_layer.layerIndex;
        this->alph = old_layer.alph; 
        this->bet = old_layer.bet;
        this->TensorFormat = old_layer.TensorFormat;
        this->DataType = old_layer.DataType;      
        this->ConvMode = old_layer.ConvMode;
        this->ActivationMode = old_layer.ActivationMode;
        if(old_layer.POOL == true)
            this->PoolingMode = old_layer.PoolingMode; 
    }

    void getConvLayerSpecs();

    void buildConvLayer(float* inbias, vector< vector<vector< vector <float> > > >& inkernal);

    void fwdProp(cudaStream_t stream, cudnnHandle_t CUDNN, float *inputTensor, float* &outputTensor, float* &conv_output, float* &poolTensor, void* &d_workspace);

};


void ConvLayers::getConvLayerSpecs(){



  checkCUDNN(cudnnCreateTensorDescriptor(&input_descriptor));
  checkCUDNN(cudnnSetTensor4dDescriptor(input_descriptor,
                                          TensorFormat,
                                          DataType,
                                          inDims.Batch,
                                          inDims.Channels, 
                                          inDims.Height, 
                                          inDims.Width));

   
  // --- Build the Kernel which is going to convolve over the input ---//
  
  
  checkCUDNN(cudnnCreateFilterDescriptor(&kernel_descriptor));
  checkCUDNN(cudnnSetFilter4dDescriptor(kernel_descriptor,
                                        DataType,
                                        TensorFormat,
                                        kernelDims.kernelSize,
                                        inDims.Channels,
                                        kernelDims.kernelHeight,
                                        kernelDims.kernelWidth));

  // --- Build the Convolution descriptor --- //

  
  checkCUDNN(cudnnCreateConvolutionDescriptor(&convolution_descriptor));
  checkCUDNN(cudnnSetConvolution2dDescriptor(convolution_descriptor,
                                            kernelDims.padHeight,
                                            kernelDims.padWidth,
                                            kernelDims.strideHeight,
                                            kernelDims.strideWidth,
                                            kernelDims.dilationHeight,
                                            kernelDims.dilationWidth,
                                            ConvMode,
                                            DataType));

  // --- This function returns the dimensions of the resulting 4D tensor of a 2D convolution,     //
  // ---given the convolution descriptor, the input tensor descriptor and the filter descriptor --- //

  checkCUDNN(cudnnGetConvolution2dForwardOutputDim(convolution_descriptor,
                                                 input_descriptor,
                                                 kernel_descriptor,
                                                 &convOutDimBatchSize,
                                                 &convOutDimChannels,
                                                 &convOutDimHeight,
                                                 &convOutDimWidth));
  
  outDims.Height = convOutDimHeight;
  outDims.Width = convOutDimWidth;
  outDims.Channels = convOutDimChannels;
  outDims.Batch = convOutDimBatchSize;
  
  checkCUDNN(cudnnCreateTensorDescriptor(&bias_descriptor));
  checkCUDNN(cudnnSetTensor4dDescriptor(bias_descriptor,
                                            TensorFormat,
                                            DataType,
                                            convOutDimBatchSize,
                                           convOutDimChannels,
                                           convOutDimHeight,
                                           convOutDimWidth));

  // ---Build the output Descriptor ---//

  
  checkCUDNN(cudnnCreateTensorDescriptor(&convOutput_descriptor));
  checkCUDNN(cudnnSetTensor4dDescriptor(convOutput_descriptor,
                                        TensorFormat,
                                        DataType, 
                                        convOutDimBatchSize,
                                        convOutDimChannels,
                                        convOutDimHeight,
                                        convOutDimWidth));

  // -- Size references for next conv layer --- //

  
  // REMOVED THIS, PUT IT IN FORWARD

  
  checkCUDNN(cudnnCreateActivationDescriptor(&activation_descriptor));
  checkCUDNN(cudnnSetActivationDescriptor(activation_descriptor,
                                        ActivationMode,
                                        CUDNN_PROPAGATE_NAN,
                                        /*relu_coef=*/0));


  /*
  Do some adjustment if the output dimension of pooling layer is not an integer (which will give an error) 
  Each dimension h and w of the output images is computed as followed:
  outputDim = 1 + (inputDim + 2*padding - windowDim)/poolingStride;
  */

  // check if the user has asked to create a pooling layer for this conv layer
  if(POOL){

    if((outDims.Height - poolDims.Height)%2 != 0){
      poolDims.Height = (poolDims.Height == 2) ? 3 : 2;
    }

    if((outDims.Width - poolDims.Width)%2 != 0){
      poolDims.Width = (poolDims.Width == 2) ? 3 : 2;
    }

    
    checkCUDNN(cudnnCreatePoolingDescriptor(&pooling_descriptor));
    checkCUDNN(cudnnSetPooling2dDescriptor(pooling_descriptor,
                                            PoolingMode,
                                            CUDNN_NOT_PROPAGATE_NAN,
                                            poolDims.Height,
                                            poolDims.Width,
                                            poolDims.padHeight,
                                            poolDims.padWidth,
                                            poolDims.strideHeight,
                                            poolDims.strideWidth));

    checkCUDNN(cudnnGetPooling2dForwardOutputDim(pooling_descriptor,
                                              convOutput_descriptor,
                                                  &poolOutBatchSize,
                                                  &poolOutChannels,
                                                  &poolOutHeight,
                                                  &poolOutWidth));

    
    checkCUDNN(cudnnCreateTensorDescriptor(&poolTensor_descriptor));  
    checkCUDNN(cudnnSetTensor4dDescriptor(poolTensor_descriptor,
                                          TensorFormat,
                                          DataType,
                                          poolOutBatchSize,
                                          poolOutChannels,
                                          poolOutHeight,
                                          poolOutWidth));

    outDims.Batch = poolOutBatchSize;
    outDims.Channels = poolOutChannels;
    outDims.Height = poolOutHeight;
    outDims.Width = poolOutWidth;

    }

    cout<<"Output image size "<<outDims.Batch<<" X "<<outDims.Height<<" X "<<outDims.Width<<" X "<<outDims.Channels<<endl;
}

void ConvLayers::buildConvLayer(float* inbias, vector< vector<vector< vector <float> > > >& inkernal){

    // Initialize bias and kernel tensors here //
    // Bias
    cudaMallocManaged(&biasTensor, outDims.Channels * outDims.Batch * sizeof(float));
    cudaMemcpy(biasTensor,inbias,outDims.Channels * outDims.Batch * sizeof(float),cudaMemcpyHostToDevice);

    float hkernel[kernelDims.kernelSize][inDims.Channels][kernelDims.kernelHeight][kernelDims.kernelWidth];

    for(int i = 0; i < kernelDims.kernelSize; i++){
      for(int j = 0; j < inDims.Channels; j++){
        for(int k = 0; k < kernelDims.kernelHeight; k++){
          for(int l = 0; l < kernelDims.kernelWidth; l++){
            hkernel[i][j][k][l] = inkernal[i][j][k][l]; 
          }
        }
      }
    }

    cudaMallocManaged(&kernelTensor, kernelDims.kernelSize * kernelDims.kernelHeight * kernelDims.kernelWidth * sizeof(float));
    cudaMemcpy(kernelTensor,hkernel,sizeof(hkernel),cudaMemcpyHostToDevice);
}


void ConvLayers::fwdProp(cudaStream_t stream, cudnnHandle_t CUDNN, float *inputTensor, float* &outputTensor, float* &conv_output, float* &poolTensor, void* &d_workspace)
{

    /**
    float* conv_output{nullptr}; // output of convolution operation
    float* poolTensor{nullptr};  // output of pooling layer, if exists
    void* d_workspace{nullptr};
    size_t workspaceBytes{0};
    **/
   // --- Determine the Convolution algorithm to be used in CNN layer ---//
  checkCUDNN(cudnnGetConvolutionForwardAlgorithm(CUDNN,
                                        input_descriptor,
                                        kernel_descriptor,
                                        convolution_descriptor,
                                        convOutput_descriptor,
                                        CUDNN_CONVOLUTION_FWD_PREFER_FASTEST,
                                        /*memoryLimitInBytes=*/0,
                                        &convolution_algorithm));

    size_t workspaceBytes{0};
    // --- Set up the memory required for the convolution --- //
    checkCUDNN(cudnnGetConvolutionForwardWorkspaceSize(CUDNN,
                                                     input_descriptor,
                                                     kernel_descriptor,
                                                     convolution_descriptor,
                                                     convOutput_descriptor,
                                                     convolution_algorithm,
                                                     &workspaceBytes));
    // --- Allocate Memory in the GPU for layer operation --- //    
    cudaMallocManaged(&d_workspace, workspaceBytes);

    int convout_bytes = convOutDimBatchSize * convOutDimChannels * convOutDimHeight * convOutDimWidth * sizeof(float);    
    // memory required for storing output of the conv operation (after adding bias)
    cudaMallocManaged(&conv_output, convout_bytes);
    cudaMemsetAsync(conv_output, 0, convout_bytes, stream);

    // set up memory for pool tensor if pool is true
    if(POOL){
      int poolSize =  outDims.Batch * outDims.Channels * outDims.Height * outDims.Width * sizeof(float);
      cudaMallocManaged(&poolTensor, poolSize); 
      cudaMemsetAsync(poolTensor, 0, poolSize, stream);
    }

    checkCUDNN(cudnnConvolutionForward(CUDNN,
                                    &alph,
                                    input_descriptor,
                                    inputTensor,
                                    kernel_descriptor,
                                    kernelTensor,
                                    convolution_descriptor,
                                    convolution_algorithm,
                                    d_workspace,
                                    workspaceBytes,
                                    &bet,
                                    convOutput_descriptor,
                                    conv_output));

    checkCUDNN(cudnnAddTensor(CUDNN, &alph, bias_descriptor,
                                biasTensor ,&bet, convOutput_descriptor, conv_output));

    checkCUDNN(cudnnActivationForward(CUDNN,
                                    activation_descriptor,
                                    &alph,
                                    convOutput_descriptor,
                                    conv_output,
                                    &bet,
                                    convOutput_descriptor,
                                    conv_output));

    if(POOL){
        checkCUDNN(cudnnPoolingForward(CUDNN,
                                    pooling_descriptor,
                                    &alph,
                                    convOutput_descriptor,
                                    conv_output,
                                    &bet,
                                    poolTensor_descriptor,
                                    poolTensor));

        cudaMallocManaged(&outputTensor,sizeof(poolTensor));
        cudaMemcpyAsync(outputTensor,poolTensor,sizeof(poolTensor),cudaMemcpyDeviceToDevice, stream);
    }
    else{

        cudaMallocManaged(&outputTensor,sizeof(conv_output));
        cudaMemcpyAsync(outputTensor,conv_output,sizeof(conv_output),cudaMemcpyDeviceToDevice, stream);  	
    }
}


__global__
void addBiasFC(int dim1,int dim2, float* bias, float* res){

  int idx = blockIdx.x * blockDim.x + threadIdx.x;

  if(idx < dim1*dim2){
    res[idx] += bias[idx];
  }
}


class FCLayers : public ConvLayers{

  public:
    // cublasHandle_t CUBLAS;
    int inDims;
    int outDims;
    int batch;
    float *weights{nullptr};
    float *dcost{nullptr};
    // float *p_act{nullptr};
    float *nabla_w{nullptr};
    float *nabla_b{nullptr};
    float* ones{nullptr};
    float* d_intermediate{nullptr};
    float* dReLU_tensor{nullptr};
    


    bool last;
    float* outputTensor{nullptr};
    cudnnTensorDescriptor_t outputTensor_descriptor;
    FCLayers(){}

    FCLayers(int inDims_, int batch,int outDims_, float alpha, float beta, cudnnActivationMode_t ActivationMode_, 
      cudnnTensorFormat_t t_format, cudnnDataType_t d_type){

      this->last = false;
      this->inDims = inDims_;
      this->batch = batch;
      this->outDims = outDims_;
      this->ActivationMode = ActivationMode_;      
      this->alph = alpha;
      this->bet = beta;
      this->DataType = d_type;
      this->TensorFormat = t_format;
    }

    void set(int inDims_, int batch,int outDims_, float alpha, float beta, cudnnActivationMode_t ActivationMode_, 
      cudnnTensorFormat_t t_format, cudnnDataType_t d_type){

      this->last = false;
      this->inDims = inDims_;
      this->batch = batch;
      this->outDims = outDims_;
      this->ActivationMode = ActivationMode_;      
      this->alph = alpha;
      this->bet = beta;
      this->DataType = d_type;
      this->TensorFormat = t_format;
    }

    void getFCLayerSpecs();
    void buildFCLayer(float* inbias, vector< vector <float> >& inkernal);
    void fwdProp(cublasHandle_t CUBLAS, cudnnHandle_t CUDNN, cudaStream_t stream, float* inputTensor, float* &outputTensor);

  private:
    int numBlocks;
    int numThreads;
};



void FCLayers::getFCLayerSpecs(){

    checkCUDNN(cudnnCreateActivationDescriptor(&activation_descriptor));
    checkCUDNN(cudnnSetActivationDescriptor(activation_descriptor,
                                    ActivationMode,
                                    CUDNN_PROPAGATE_NAN,
                                    0));

    checkCUDNN(cudnnCreateTensorDescriptor(&outputTensor_descriptor));
    checkCUDNN(cudnnSetTensor4dDescriptor(outputTensor_descriptor,
                                      TensorFormat,
                                      DataType,
                                      batch, outDims, 1, 1));


}


void FCLayers::buildFCLayer(float* inbias, vector< vector <float> >& inkernal){

  // Initialization of weight matrix

  
  float *hweights;
  hweights = (float*)malloc(sizeof(float)*inDims*outDims);
  for(int i = 0; i < outDims; i++){
          for(int j=0;j<inDims;j++)
   hweights[i*outDims + j] = inkernal[i][j];
  }

  cudaMallocManaged(&weights, inDims*outDims*sizeof(float));
  cudaMemcpy(weights, hweights, inDims * outDims * sizeof(float),cudaMemcpyHostToDevice);

  free(hweights);
  // initialization of bias vector
  cudaMallocManaged(&biasTensor,outDims*batch*sizeof(float));
  cudaMemcpy(biasTensor,inbias,outDims*batch*sizeof(float),cudaMemcpyHostToDevice);
    

  
  // cudaMemset(outputTensor,0,outDims*batch*sizeof(float)); ----- If not using cudaMemset(),
  // ensure that while performing any operation on it, its multiplicatio coeff is 0, like in cublasSgemm() below, bet is 0 for the same reason

  // Decide the number of threads and blocks based on the size of the output of the FC layer (before adding the bias) for addBiasFC kernel
  if(batch*outDims <= MAX_THREADS_PER_BLOCK){
    numThreads = batch*outDims;
    numBlocks = 1;
  }
  else{
    numBlocks = roundUp(batch*outDims,MAX_THREADS_PER_BLOCK);
    numThreads = MAX_THREADS_PER_BLOCK;
  }
    cout<<"Output image size "<<outDims<<endl;
}


void FCLayers::fwdProp(cublasHandle_t CUBLAS, cudnnHandle_t CUDNN, cudaStream_t stream, float* inputTensor, float* &outputTensor){

  cudaMallocManaged(&outputTensor,outDims*batch*sizeof(float));

  cublasSgemm(CUBLAS,
        CUBLAS_OP_T,CUBLAS_OP_N,
        outDims, batch, inDims,
        &alph,
        weights,inDims,
        inputTensor,inDims,
        &bet,
        outputTensor,outDims);
  
  addBiasFC<<<numBlocks,numThreads,0,stream>>>(outDims,batch,biasTensor,outputTensor);
  
  checkCUDNN(cudnnActivationForward(CUDNN,
                  activation_descriptor,
                  &alph,
                  outputTensor_descriptor,
                  outputTensor,
                  &bet,
                  outputTensor_descriptor,
                  outputTensor));
  

}


ConvLayers convlayer1;
ConvLayers convlayer2;
ConvLayers convlayer3;
ConvLayers convlayer4;
ConvLayers convlayer5;
FCLayers fclayer1;
FCLayers fclayer2;
FCLayers fclayer3;

class Kernel{

  public :
    cudaStream_t stream;
    cublasHandle_t cublas;
    cudnnHandle_t cudnn;
    int id;
    int layer;
    double tbRatio;
    float ****fourDInput;
    float ****fourDOutput;
    float ** twoDInput;
    float ** twoDOutput;
    double execTime;
    double compTime;
    

    Kernel(){}
    
    Kernel(cudaStream_t stream,
        cublasHandle_t cublas,
        cudnnHandle_t cudnn,
        int layerIndex,
        float ****fourDInput,
        double startTime,
        double deadTime,
        double tbratio,
        int id){
            this->stream = stream;
            this->cublas = cublas;
            this->cudnn = cudnn;
            this->layer = layerIndex;
            this->fourDInput = fourDInput;
            this->execTime = deadTime - startTime;
            this->compTime = LAYERS*CONST_TIME;
            this->tbRatio = tbratio;
            this->id = id;
        }
    
    Kernel(cudaStream_t stream,
        cublasHandle_t cublas,
        cudnnHandle_t cudnn,
        int layerIndex,
        float **twoDInput,
        double startTime,
        double deadTime,
        double tbratio,
        int id){
            this->stream = stream;
            this->cublas = cublas;
            this->cudnn = cudnn;
            this->layer = layerIndex;
            this->twoDInput = twoDInput;
            this->execTime = deadTime - startTime;
            this->compTime = LAYERS*CONST_TIME;
            this->tbRatio = tbratio;
            this->id =id;
        }
    
    void update()
    {
        layer++;
        compTime -= CONST_TIME;
        // TODO update tbratio properly
        if(layer >= 7){
            tbRatio = 0.2;
            twoDInput = twoDOutput;
        }
        else{
            fourDInput = fourDOutput;
        }
    }


};

void execute(vector<Kernel> C);


typedef pair<int, Kernel> Node;

struct cmp
{
	bool operator()(const Node &A,const Node &B) const
	{
		if(A.first != B.first)
            return A.first < B.first;
        else
            return A.second.execTime - A.second.compTime < B.second.execTime - B.second.compTime;
	}
};

typedef set<Node, cmp> SET;

bool compareKernel(Kernel k1, Kernel k2){
    return k1.compTime < k2.compTime;
}

int getSlackTime(int compTime, int execTime){
    return execTime - compTime;
}



vector<Kernel> G;

SET enqueue(SET& Queue, Kernel k){
    SET newQueue;
    newQueue.insert({getSlackTime(k.compTime, k.execTime), k});
    for(SET::iterator it=Queue.begin();it!=Queue.end();it++){
        Kernel temp_kernel = it->second;
        temp_kernel.execTime -= 1;
        newQueue.insert({getSlackTime(temp_kernel.compTime,temp_kernel.execTime),temp_kernel});
    }
    Queue.clear();
    return newQueue;
}


SET dequeue(SET& Queue){
    SET newQueue;
    
    cout<<"Printing Queue"<<endl;
    for(SET:: iterator it=Queue.begin();it!=Queue.end();it++){
        Kernel temp_kernel = it->second;
        cout<<"ID: "<<temp_kernel.id<<endl;
        temp_kernel.execTime -= CONST_TIME;
        newQueue.insert({getSlackTime(temp_kernel.compTime,temp_kernel.execTime),temp_kernel});
    }
    Queue.clear();
    vector<Kernel> C;
    if(G.size()>0){
        execute(G);
        cout<<"Execution size "<<G.size()<<endl;
    }
        
    for(int i=0;i<G.size();i++){
        cout<<"EXEC Layer: "<<G[i].layer<<endl;
        cout<<"Priority:"<<G[i].execTime - G[i].compTime<<endl;
        if(G[i].layer != LAYERS){
            G[i].update();
            newQueue = enqueue(newQueue, G[i]);
        }
    }
    G.clear();
    Kernel temp = (*(newQueue.begin())).second;
    newQueue.erase(newQueue.begin());
    G.push_back(temp);
    double total = 0;
    total += temp.tbRatio;
    if(total < 1){
        for(SET::iterator it = newQueue.begin();it!=newQueue.end();){
            if((*it).second.tbRatio<1){
                Kernel temp2 = (*it).second;
                total += temp2.tbRatio;
                it = newQueue.erase(it);
                G.push_back(temp2);
            }
            else
            {
                it++;
            }
            cout<<"Total TB "<<total<<endl;
            if(total > 1)
                break;
        }
    }
    else
    {
        cout<<"Single kernel execution"<<endl;
        execute(G);
        for(int i=0;i<G.size();i++){
            cout<<"Priority:"<<G[i].execTime - G[i].compTime<<endl;
            if(G[i].layer != LAYERS){
                G[i].update();
                newQueue = enqueue(newQueue, G[i]);
            }
        }
        G.clear();

        return newQueue;
    }
    //lookahead to be implemented


    execute(G);
    cout<<"Execution size: "<<G.size()<<endl;
    for(int i=0;i<G.size();i++){
        cout<<"Priority:"<<G[i].execTime - G[i].compTime<<endl;
        if(G[i].layer != LAYERS){
            G[i].update();
            newQueue = enqueue(newQueue, G[i]);
        }
    }
    G.clear();

    return newQueue;   
}


void call_fc1(FCLayers &fclayer1, cudaStream_t stream, cudnnHandle_t cudnn, cublasHandle_t cublas, float ****& input, float** &output)
{
    float *input_layer1;
    convDim_t InputDims = setConvSpecs(7, 7, 256, BATCH_SIZE);
    cudaMallocManaged(&input_layer1, InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float));
    cudaMemcpyAsync(input_layer1, input, InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float), cudaMemcpyHostToDevice, stream);

    float *outputTensor;
    
    fclayer1.fwdProp(cublas, cudnn, stream, input_layer1, outputTensor);

    int out_size =  4096*BATCH_SIZE;

    output = (float **) malloc(BATCH_SIZE*sizeof(float*));
    for(int i=0;i<BATCH_SIZE;++i) output[i] = (float *) malloc(4096*sizeof(float));
    for(int i=0;i<BATCH_SIZE;++i){
        for(int j=0;j<4096;++j){
            output[i][j] = 0;
        }
    }


    cudaMemcpyAsync(output, outputTensor, out_size*sizeof(float), cudaMemcpyDeviceToHost, stream);
    cout << "FC1 DONE" << endl;
}


void call_fc2(FCLayers &fclayer2, cudaStream_t stream, cudnnHandle_t cudnn, cublasHandle_t cublas, float **& input, float** &output)
{
    float *input_layer2;
    int InputDims = 4096;
    cudaMallocManaged(&input_layer2, InputDims * BATCH_SIZE * sizeof(float));
    cudaMemcpyAsync(input_layer2, input, InputDims * BATCH_SIZE * sizeof(float), cudaMemcpyHostToDevice, stream);

    float * outputTensor; 
    fclayer2.fwdProp(cublas, cudnn, stream, input_layer2, outputTensor);

    int out_size =  4096*BATCH_SIZE;
 
    output = (float **) malloc(BATCH_SIZE*sizeof(float*));
    for(int i=0;i<BATCH_SIZE;++i) output[i] = (float *) malloc(4096*sizeof(float));
    for(int i=0;i<BATCH_SIZE;++i){
        for(int j=0;j<4096;++j){
            output[i][j] = 0;
        }
    }

    cudaMemcpyAsync(output, outputTensor, out_size*sizeof(float), cudaMemcpyDeviceToHost, stream);
    cout << "FC2 DONE" << endl;
}


void call_fc3(FCLayers &fclayer3, cudaStream_t stream, cudnnHandle_t cudnn, cublasHandle_t cublas, float **& input, float** &output)
{

    float *input_layer3;
    int InputDims = 4096;
    cudaMallocManaged(&input_layer3, InputDims * BATCH_SIZE * sizeof(float));
    cudaMemcpyAsync(input_layer3, input, InputDims * BATCH_SIZE * sizeof(float), cudaMemcpyHostToDevice, stream);

    float * outputTensor;
    fclayer3.fwdProp(cublas, cudnn, stream, input_layer3, outputTensor);

    int out_size =  1000*BATCH_SIZE;
    cout<<"HI2"<<endl;
    output = (float **) malloc(BATCH_SIZE*sizeof(float*));
    cout<<"HI3"<<endl;
    for(int i=0;i<BATCH_SIZE;++i) output[i] = (float *) malloc(1000*sizeof(float));
    for(int i=0;i<BATCH_SIZE;++i){
        for(int j=0;j<1000;++j){
            output[i][j] = 0;
        }
    }
    cout<<"HI4"<<endl;
    cudaMemcpyAsync(output, outputTensor, out_size*sizeof(float), cudaMemcpyDeviceToHost, stream);
    cout << "FC3 DONE" << endl;
}



void fill_vec(vector<vector<vector<vector<float> > > > &ker, float bia[], string file_loc, int x, int y, int z, int w){
  
  std::ifstream File(file_loc);
  std::string Str;
  int b_c=0,i=0,j=0,k=0,l=0,count=0,length=y*z*w+1;
  while (std::getline(File, Str))
  {
      istringstream ss(Str);
      string word;
      while (ss >> word) 
      {
          if(count%length==0){
              bia[b_c++]=stof(word);
          }
          else{
              ker[i][j][k][l++]=stof(word);
              if(l==w)k++,l=0;
              if(k==z)j++,k=0;
              if(j==y)i++,j=0;
          }
          count++;
          //cout << word << "\n";
      }
  }
  //cout<<count<<'\n';
  
  File.close();
}

void fill_vec(vector<vector<float> > &ker, float bia[], string file_loc, int x, int y){
  
  std::ifstream File(file_loc);
  std::string Str;
  int b_c=0,i=0,j=0,count=0,length=y+1;
  while (std::getline(File, Str))
  {
      istringstream ss(Str);
      string word;
      while (ss >> word) 
      {
          if(count%length==0){
              bia[b_c++]=stof(word);
          }
          else{
              ker[i][j++]=stof(word);
              if(j==y)i++,j=0;
          }
          count++;
          //cout << word << "\n";
      }
  }
  //cout<<count<<'\n';
  
  File.close();
}

void fill_vec_special(vector<vector<float> > &ker, float bia[], string file_loc, int x, int y){
  
  std::ifstream File(file_loc);
  std::string Str;
  int b_c=0,i=0,j=0,j_fill=0,count=0,length=y+1;
  while (std::getline(File, Str))
  {
      istringstream ss(Str);
      string word;
      while (ss >> word) 
      {
          if(count%length==0){
              bia[b_c++]=stof(word);
          }
          else{
              ker[i][j_fill++]=stof(word);
              j++;
              if(j%36==0){
                  j_fill+=13;
              }
              if(j_fill==y)i++,j_fill=0,j=0;
          }
          count++;
          //cout << word << "\n";
      }
  }
  //cout<<count<<'\n';
  
  File.close();
}

float **** MemAlloc(int batch, int channel, int height, int width){
    float ****output_feature;
    output_feature = (float****)malloc(batch*sizeof(float***));

    for(int i=0;i<batch;++i)
    {
        output_feature[i] = (float***)malloc(channel*sizeof(float**));
        for(int j=0;j<channel;++j)
        {
            output_feature[i][j] = (float**)malloc(height*sizeof(float*));
            for(int k=0;k<height;++k)
            {
                output_feature[i][j][k] = (float*)malloc(width*sizeof(float));
            }
        }
    }

    for(int i=0;i<batch;++i)
    {
        for(int j=0;j<channel;++j)
        {
            for(int k=0;k<height;++k)
            {
                for(int l=0;l<width;++l)
                {
                    output_feature[i][j][k][l]=0;
                }
            }
        }
    }
    return output_feature;
}

void call_conv1(ConvLayers &convlayer1, cudaStream_t stream, cudnnHandle_t CUDNN, float**** &input, float**** &output)
{
    float *input_layer1;
    convDim_t firstLayerInputDims = setConvSpecs(227, 227, 3, BATCH_SIZE);
    cudaMallocManaged(&input_layer1,firstLayerInputDims.Height * firstLayerInputDims.Width * firstLayerInputDims.Channels * firstLayerInputDims.Batch * sizeof(float));
    cudaMemcpyAsync(input_layer1,input,227*227*3* sizeof(float),cudaMemcpyHostToDevice, stream);
    float *outputTensor, *conv_output, *poolTensor;
    void *d_workspace;

    convlayer1.fwdProp(stream, CUDNN, input_layer1, outputTensor, conv_output, poolTensor, d_workspace);

    int out_size =  convlayer1.outDims.Height*convlayer1.outDims.Width*convlayer1.outDims.Channels*convlayer1.outDims.Batch;
    output = MemAlloc(convlayer1.outDims.Batch, convlayer1.outDims.Channels, convlayer1.outDims.Height, convlayer1.outDims.Width);
    cudaMemcpyAsync(output, outputTensor, out_size*sizeof(float), cudaMemcpyDeviceToHost, stream);
    cout << "CONV1 DONE" << endl;
}

void call_conv2(ConvLayers &convlayer2, cudaStream_t stream, cudnnHandle_t CUDNN, float ****& input, float**** &output)
{
    float *input_layer2;
    convDim_t InputDims = setConvSpecs(28, 28, 64, BATCH_SIZE);
    cudaMallocManaged(&input_layer2,InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float));
    cudaMemcpyAsync(input_layer2,input,InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float), cudaMemcpyHostToDevice, stream);
    float *outputTensor, *conv_output, *poolTensor;
    void *d_workspace;

    convlayer2.fwdProp(stream, CUDNN, input_layer2, outputTensor, conv_output, poolTensor, d_workspace);

    int out_size =  convlayer2.outDims.Height*convlayer2.outDims.Width*convlayer2.outDims.Channels*convlayer2.outDims.Batch;
    output = MemAlloc(convlayer2.outDims.Batch, convlayer2.outDims.Channels, convlayer2.outDims.Height, convlayer2.outDims.Width);

    cudaMemcpyAsync(output, outputTensor, out_size*sizeof(float), cudaMemcpyDeviceToHost, stream);
    cout << "CONV2 DONE" << endl;
}

void call_conv3(ConvLayers &convlayer3, cudaStream_t stream, cudnnHandle_t CUDNN, float ****& input, float**** &output)
{
    float *input_layer3;
    convDim_t InputDims = setConvSpecs(14, 14, 192, BATCH_SIZE);
    cudaMallocManaged(&input_layer3,InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float));
    cudaMemcpyAsync(input_layer3,input,InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float), cudaMemcpyHostToDevice, stream);
    float *outputTensor, *conv_output, *poolTensor;
    void *d_workspace;

    convlayer3.fwdProp(stream, CUDNN, input_layer3, outputTensor, conv_output, poolTensor, d_workspace);

    int out_size =  convlayer3.outDims.Height*convlayer3.outDims.Width*convlayer3.outDims.Channels*convlayer3.outDims.Batch;
    output = MemAlloc(convlayer3.outDims.Batch, convlayer3.outDims.Channels, convlayer3.outDims.Height, convlayer3.outDims.Width);

    cudaMemcpyAsync(output, outputTensor, out_size*sizeof(float), cudaMemcpyDeviceToHost, stream);
    cout << "CONV3 DONE" << endl;
}

void call_conv4(ConvLayers &convlayer4, cudaStream_t stream, cudnnHandle_t CUDNN, float ****& input, float**** &output)
{
    float *input_layer4;
    convDim_t InputDims = setConvSpecs(14, 14, 384, BATCH_SIZE);
    cudaMallocManaged(&input_layer4,InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float));
    cudaMemcpyAsync(input_layer4,input,InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float), cudaMemcpyHostToDevice, stream);
    float *outputTensor, *conv_output, *poolTensor;
    void *d_workspace;

    convlayer4.fwdProp(stream, CUDNN, input_layer4, outputTensor, conv_output, poolTensor, d_workspace);

    int out_size =  convlayer4.outDims.Height*convlayer4.outDims.Width*convlayer4.outDims.Channels*convlayer4.outDims.Batch;
    output = MemAlloc(convlayer4.outDims.Batch, convlayer4.outDims.Channels, convlayer4.outDims.Height, convlayer4.outDims.Width);

    cudaMemcpyAsync(output, outputTensor, out_size*sizeof(float), cudaMemcpyDeviceToHost, stream);
    cout << "CONV4 DONE" << endl;
}

void call_conv5(ConvLayers &convlayer5, cudaStream_t stream, cudnnHandle_t CUDNN, float ****& input, float**** &output)
{
    float *input_layer5;
    convDim_t InputDims = setConvSpecs(28, 28, 64, BATCH_SIZE);
    cudaMallocManaged(&input_layer5,InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float));
    cudaMemcpyAsync(input_layer5,input,InputDims.Height * InputDims.Width * InputDims.Channels * InputDims.Batch * sizeof(float), cudaMemcpyHostToDevice, stream);
    float *outputTensor, *conv_output, *poolTensor;
    void *d_workspace;

    convlayer5.fwdProp(stream, CUDNN, input_layer5, outputTensor, conv_output, poolTensor, d_workspace);

    int out_size =  convlayer5.outDims.Height*convlayer5.outDims.Width*convlayer5.outDims.Channels*convlayer5.outDims.Batch;
    output = MemAlloc(convlayer5.outDims.Batch, convlayer5.outDims.Channels, convlayer5.outDims.Height, convlayer5.outDims.Width);

    cudaMemcpyAsync(output, outputTensor, out_size*sizeof(float), cudaMemcpyDeviceToHost, stream);
    cout << "CONV5 DONE" << endl;
}

//  execute all the kernels in the given vector
void execute(vector<Kernel> C){
    for(int i=0;i<C.size();i++){
        Kernel K = C[i];
        if(K.layer == 1)
            call_conv1(convlayer1, K.stream, K.cudnn, K.fourDInput, K.fourDOutput);
        else if(K.layer == 2)
            call_conv2(convlayer2, K.stream, K.cudnn, K.fourDInput, K.fourDOutput);
        else if(K.layer == 3)
            call_conv3(convlayer3, K.stream, K.cudnn, K.fourDInput, K.fourDOutput);
        else if(K.layer == 4)
            call_conv4(convlayer4, K.stream, K.cudnn, K.fourDInput, K.fourDOutput);
        else if(K.layer == 5)
            call_conv5(convlayer5, K.stream, K.cudnn, K.fourDInput, K.fourDOutput);
        else if(K.layer == 6)
            call_fc1(fclayer1, K.stream, K.cudnn, K.cublas, K.fourDInput, K.twoDOutput);
        else if(K.layer == 7)
            call_fc2(fclayer2, K.stream, K.cudnn, K.cublas, K.twoDInput, K.twoDOutput);       
        else
            call_fc3(fclayer3, K.stream, K.cudnn, K.cublas, K.twoDInput, K.twoDOutput);
    }
    cudaDeviceSynchronize();
}


void FreeMem(float ****output_feature, int batch, int channel, int height, int width)
{  
    
    for(int i=0;i<batch;++i)
    {
        for(int j=0;j<channel;++j)
        {
            for(int k=0;k<height;++k)
            {
                free(output_feature[i][j][k]);
            }
            free(output_feature[i][j]);
        }
        free(output_feature[i]);
    }
}

//channel, height, width
	int main(){
    float ****img1 = MemAlloc(BATCH_SIZE, 3, 227, 227);
    for(int i=0;i<BATCH_SIZE;++i){
        for(int j=0; j<3;++j){
            for(int k=0;k<227;++k){
                for(int l=0;l<227;++l){
                    img1[i][j][k][l]=1;
                }
            }
        }
    }

    float ****img2 = MemAlloc(BATCH_SIZE, 3, 227, 227);
    for(int i=0;i<BATCH_SIZE;++i){
        for(int j=0; j<3;++j){
            for(int k=0;k<227;++k){
                for(int l=0;l<227;++l){
                    img2[i][j][k][l]=1;
                }
            }
        }
    }
	//float img[3][227][227]={0.0};

	vector< vector<vector< vector <float> > > > 
		conv1(64,vector<vector< vector <float> > >(3,vector< vector <float> >(11,vector <float>(11,0.0)))),
		conv2(192,vector<vector< vector <float> > >(64,vector< vector <float> >(5,vector <float>(5,0.0)))),
		conv3(384,vector<vector< vector <float> > >(192,vector< vector <float> >(3,vector <float>(3,0.0)))),
		conv4(256,vector<vector< vector <float> > >(384,vector< vector <float> >(3,vector <float>(3,0.0)))),
		conv5(256,vector<vector< vector <float> > >(256,vector< vector <float> >(3,vector <float>(3,0.0))))
	;

	vector< vector <float> > 
		lin6(4096,  vector <float> (12544, 0.0)),
		lin7(4096,  vector <float> (4096, 0.0)),
		lin8(1000,  vector <float> (4096, 0.0));
	//9216 appended with zeroes from 36 to 49
	float bias1[64]={0.0}, bias2[192]={0.0}, bias3[384]={0.0}, bias4[256]={0.0}, bias5[256]={0.0},bias6[4096]={0.0}, bias7[4096]={0.0}, bias8[1000]={0.0};

	fill_vec(conv1, bias1, "/content/drive/MyDrive/HP3/feature1.txt", 64, 3, 11, 11);
	fill_vec(conv2, bias2, "/content/drive/MyDrive/HP3/feature2.txt", 192, 64, 5, 5);
	fill_vec(conv3, bias3, "/content/drive/MyDrive/HP3/feature3.txt", 384, 192, 3, 3);
	fill_vec(conv4, bias4, "/content/drive/MyDrive/HP3/feature4.txt", 256, 384, 3, 3);
	fill_vec(conv5, bias5, "/content/drive/MyDrive/HP3/feature5.txt", 256, 256, 3, 3);

	fill_vec_special(lin6, bias6, "/content/drive/MyDrive/HP3/feature7.txt", 4096, 12544);

	fill_vec(lin7, bias7, "/content/drive/MyDrive/HP3/feature7.txt", 4096, 4096);
	fill_vec(lin8, bias8, "/content/drive/MyDrive/HP3/feature8.txt", 1000, 4096);

    poolDim_t poolDim1 = setPoolSpecs((bool)OVERLAP_POOLING);
	convDim_t firstLayerInputDims = setConvSpecs(227, 227, 3, BATCH_SIZE);
	kernelDim_t layerKernel1 = setKernelSpecs(64,11,11,4,4,2,2,1,1);
    convlayer1.set(1, firstLayerInputDims, layerKernel1, poolDim1, alpha, beta, CUDNN_TENSOR_NHWC, CUDNN_DATA_FLOAT, CUDNN_CROSS_CORRELATION,
						CUDNN_ACTIVATION_RELU, CUDNN_POOLING_MAX);
	convlayer1.getConvLayerSpecs();
	convlayer1.buildConvLayer(bias1, conv1);



	poolDim_t poolDim2 = setPoolSpecs((bool)OVERLAP_POOLING);
	kernelDim_t layerKernel2 = setKernelSpecs(192,5,5,1,1,2,2,1,1);
	convlayer2.set(2, convlayer1.outDims, layerKernel2, poolDim2, alpha, beta, CUDNN_TENSOR_NHWC, CUDNN_DATA_FLOAT, CUDNN_CROSS_CORRELATION,
						CUDNN_ACTIVATION_RELU, CUDNN_POOLING_MAX);
	convlayer2.getConvLayerSpecs();
	convlayer2.buildConvLayer(bias2, conv2);

	// poolDim_t poolDim2 = setPoolSpecs((bool)OVERLAP_POOLING);
	kernelDim_t layerKernel3 = setKernelSpecs(384,3,3,1,1,1,1,1,1);
	convlayer3.set(3, convlayer2.outDims, layerKernel3, alpha, beta, CUDNN_TENSOR_NHWC, CUDNN_DATA_FLOAT, CUDNN_CROSS_CORRELATION,
						CUDNN_ACTIVATION_RELU);
	convlayer3.getConvLayerSpecs();
	convlayer3.buildConvLayer(bias3, conv3);


	// poolDim_t poolDim2 = setPoolSpecs((bool)OVERLAP_POOLING);
	kernelDim_t layerKernel4 = setKernelSpecs(256,3,3,1,1,1,1,1,1);
	convlayer4.set(4, convlayer3.outDims, layerKernel4, alpha, beta, CUDNN_TENSOR_NHWC, CUDNN_DATA_FLOAT, CUDNN_CROSS_CORRELATION,
						CUDNN_ACTIVATION_RELU);
	convlayer4.getConvLayerSpecs();
	convlayer4.buildConvLayer(bias4, conv4);


	poolDim_t poolDim5 = setPoolSpecs((bool)OVERLAP_POOLING);
	kernelDim_t layerKernel5 = setKernelSpecs(256,3,3,1,1,1,1,1,1);
	convlayer5.set(5, convlayer4.outDims, layerKernel5, poolDim5, alpha, beta, CUDNN_TENSOR_NHWC, CUDNN_DATA_FLOAT, CUDNN_CROSS_CORRELATION,
						CUDNN_ACTIVATION_RELU, CUDNN_POOLING_MAX);
	convlayer5.getConvLayerSpecs();
	convlayer5.buildConvLayer(bias5, conv5); 

    fclayer1.set(7*7*256, BATCH_SIZE, 4096 , alpha, beta, CUDNN_ACTIVATION_RELU,CUDNN_TENSOR_NHWC,
          CUDNN_DATA_FLOAT);
    fclayer1.getFCLayerSpecs();
    fclayer1.buildFCLayer(bias6, lin6);


    fclayer2.set(fclayer1.outDims, BATCH_SIZE, 4096 , alpha, beta, CUDNN_ACTIVATION_RELU,CUDNN_TENSOR_NHWC,
            CUDNN_DATA_FLOAT);
    fclayer2.getFCLayerSpecs();
    fclayer2.buildFCLayer(bias7, lin7);


    fclayer3.set(fclayer2.outDims, BATCH_SIZE, 1000 , alpha, beta, CUDNN_ACTIVATION_RELU,CUDNN_TENSOR_NHWC,
          CUDNN_DATA_FLOAT);
    fclayer3.getFCLayerSpecs();
    fclayer3.buildFCLayer(bias8, lin8);
	cout<<"BUILT ALL LAYERS"<<endl;

    cudaStream_t stream1, stream2;
    cudaStreamCreate(&stream1);
    cudaStreamCreate(&stream2);

    cudnnHandle_t cudnn1, cudnn2;
	checkCUDNN(cudnnCreate(&cudnn1));
    cudnnSetStream(cudnn1, stream1);

    checkCUDNN(cudnnCreate(&cudnn2));
    cudnnSetStream(cudnn2, stream2);

	cublasHandle_t cublas1, cublas2;
	cublasCreate(&cublas1);
    cublasCreate(&cublas2);

    cublasSetStream(cublas1, stream1);
    cublasSetStream(cublas2, stream2);


    SET pq;
    Kernel k1(stream1,
        cublas1,
        cudnn1,
        1,
        img1,
        0,
        50,
        0.6,
        1);

    cout<<"K1:"<<k1.tbRatio<<" "<<k1.compTime<<" "<<k1.execTime<<endl;

    Kernel k2(stream1,
        cublas1,
        cudnn1,
        1,
        img1,
        0,
        30,
        0.6,
        2);

    Kernel k3(stream1,
        cublas1,
        cudnn1,
        1,
        img1,
        0,
        40,
        0.3,
        3);
    
    for(int i=0;i<100;i++){
        Kernel k(stream1,
            cublas1,
            cudnn1,
            1,
            img1,
            0,
            50,
            0.6,
            i);

        pq = enqueue(pq, k);
    }

    while(!pq.empty()){
        pq = dequeue(pq);
    }


    //cublasSetStream(cublas2, stream2);

    // float ****output11, ****output12, ****output13, ****output14, ****output15, **output16,**output17, **output18;
    // //float ****output21, ****output22, ****output23, ****output24, ****output25, **output26,**output27, **output28;

    // call_conv1(convlayer1, stream1, cudnn1, img1, output11);
    // call_conv2(convlayer2, stream1, cudnn1, output11, output12);
    // call_conv3(convlayer3, stream1, cudnn1, output12, output13);
    // call_conv4(convlayer4, stream1, cudnn1, output13, output14);
    // call_conv5(convlayer5, stream1, cudnn1, output14, output15);
    // cudaStreamSynchronize(stream1);
    // FreeMem(output11, 1, 64, 28, 28);
    // FreeMem(output12, 1, 192, 14, 14);
    // FreeMem(output13, 1, 384, 14, 14);
    // FreeMem(output14, 1, 256, 14, 14);

    // call_fc1(fclayer1, stream1, cudnn1,  cublas1, output15, output16);
    // //call_fc2(fclayer2, stream1, cudnn1,  cublas1, output16, output17);
    // cudaStreamSynchronize(stream1);

    // call_fc3(fclayer3, stream1, cudnn1,  cublas1, output16, output18);
    // cudaStreamSynchronize(stream1);
    /**
    call_conv1(convlayer1, stream1, cudnn1, img2, output21);
    call_conv2(convlayer2, stream1, cudnn1, output21, output22);
    call_conv3(convlayer3, stream1, cudnn1, output22, output23);
    call_conv4(convlayer4, stream1, cudnn1, output23, output24);
    call_conv5(convlayer5, stream1, cudnn1, output24, output25);
    call_fc1(fclayer1, stream1, cudnn1,  cublas1, output25, output26);
    call_fc2(fclayer2, stream1, cudnn1,  cublas1, output26, output27);
    call_fc3(fclayer3, stream1, cudnn1,  cublas1, output27, output28);
    **/
    // cout<<output18[0][0]<<endl;

    return 0;
}



'File written in /content/src/alex.cu'

In [ ]:
!nvcc /content/src/alex.cu -o /content/src/hellocuda

/tmp/tmpxft_00000298_00000000-11_alex.o: In function `ConvLayers::getConvLayerSpecs()':
tmpxft_00000298_00000000-6_alex.cudafe1.cpp:(.text+0x45b): undefined reference to `cudnnCreateTensorDescriptor'
tmpxft_00000298_00000000-6_alex.cudafe1.cpp:(.text+0x4a0): undefined reference to `cudnnGetErrorString'
tmpxft_00000298_00000000-6_alex.cudafe1.cpp:(.text+0x50e): undefined reference to `cudnnSetTensor4dDescriptor'
tmpxft_00000298_00000000-6_alex.cudafe1.cpp:(.text+0x557): undefined reference to `cudnnGetErrorString'
tmpxft_00000298_00000000-6_alex.cudafe1.cpp:(.text+0x593): undefined reference to `cudnnCreateFilterDescriptor'
tmpxft_00000298_00000000-6_alex.cudafe1.cpp:(.text+0x5d8): undefined reference to `cudnnGetErrorString'
tmpxft_00000298_00000000-6_alex.cudafe1.cpp:(.text+0x646): undefined reference to `cudnnSetFilter4dDescriptor'
tmpxft_00000298_00000000-6_alex.cudafe1.cpp:(.text+0x68f): undefined reference to `cudnnGetErrorString'
tmpxft_00000298_00000000-6_alex.cudafe1.cpp:(.text

In [ ]:
!/content/src/hellocuda

/bin/bash: /content/src/hellocuda: No such file or directory


In [ ]:
%%cuda --name hello.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void vectorAdd(const float *A, const float *B, float *C, int numElements)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;

    if (i < numElements)
    {
        C[i] = A[i] + B[i];
    }
}

int main(void)
{
    // Error code to check return values for CUDA calls
    cudaError_t err = cudaSuccess;

    // Print the vector length to be used, and compute its size
    int numElements = 0;
    scanf("%d",&numElements);
    
    size_t size = numElements * sizeof(float);
    printf("[Vector addition of %d elements]\n", numElements);

    // Allocate the host input vector A
    float *h_A = (float *)malloc(size);

    // Allocate the host input vector B
    float *h_B = (float *)malloc(size);

    // Allocate the host output vector C
    float *h_C = (float *)malloc(size);

    // Verify that allocations succeeded
    if (h_A == NULL || h_B == NULL || h_C == NULL)
    {
        fprintf(stderr, "Failed to allocate host vectors!\n");
        exit(EXIT_FAILURE);
    }

    // Initialize the host input vectors
    for (int i = 0; i < numElements; ++i)
    {
        h_A[i] = rand()/(float)RAND_MAX;
        h_B[i] = rand()/(float)RAND_MAX;
    }

    // Allocate the device input vector A
    float *d_A = NULL;
    err = cudaMalloc((void **)&d_A, size);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to allocate device vector A (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Allocate the device input vector B
    float *d_B = NULL;
    err = cudaMalloc((void **)&d_B, size);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to allocate device vector B (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Allocate the device output vector C
    float *d_C = NULL;
    err = cudaMalloc((void **)&d_C, size);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to allocate device vector C (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Copy the host input vectors A and B in host memory to the device input vectors in
    // device memory
    printf("Copy input data from the host memory to the CUDA device\n");
    err = cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to copy vector A from host to device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    err = cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to copy vector B from host to device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Launch the Vector Add CUDA Kernel
    int threadsPerBlock = 256;
    int blocksPerGrid =(numElements + threadsPerBlock - 1) / threadsPerBlock;
    printf("CUDA kernel launch with %d blocks of %d threads\n", blocksPerGrid, threadsPerBlock);
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, numElements);
    err = cudaGetLastError();

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to launch vectorAdd kernel (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Copy the device result vector in device memory to the host result vector
    // in host memory.
    printf("Copy output data from the CUDA device to the host memory\n");
    err = cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to copy vector C from device to host (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Verify that the result vector is correct
    for (int i = 0; i < numElements; ++i)
    {
        if (fabs(h_A[i] + h_B[i] - h_C[i]) > 1e-5)
        {
            fprintf(stderr, "Result verification failed at element %d!\n", i);
            exit(EXIT_FAILURE);
        }
    }

    printf("Test PASSED\n");

    // Free device global memory
    err = cudaFree(d_A);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to free device vector A (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    err = cudaFree(d_B);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to free device vector B (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    err = cudaFree(d_C);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to free device vector C (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    // Reset the device and exit
    // cudaDeviceReset causes the driver to clean up all state. While
    // not mandatory in normal operation, it is good practice.  It is also
    // needed to ensure correct operation when the application is being
    // profiled. Calling cudaDeviceReset causes all profile data to be
    // flushed before the application exits
    err = cudaDeviceReset();

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to deinitialize the device! error=%s\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    printf("Done\n");
    return 0;
}

'File written in /content/src/hello.cu'

In [ ]:
!nvcc src/hello.cu -o src/hello

In [ ]:
!src/hello

100
[Vector addition of 100 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 1 blocks of 256 threads
Copy output data from the CUDA device to the host memory
Test PASSED
Done
